In [1]:
import numpy as np
from scipy import ndimage as ndi
from skimage.io import imread
from skimage.measure import label, find_contours
from skimage.morphology import square, binary_dilation
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from matplotlib import pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed, cpu_count
from typing import Tuple
import tifffile as tf

In [2]:
def edt_from_center_of_mass(img2d: np.ndarray) -> np.ndarray:
    assert 2 == len(img2d.shape), img2d.shape
    if 0 == img2d.sum():
        return img2d
    cx, cy = ndi.center_of_mass(img2d)
    center_mask = np.ones(img2d.shape[-2:])
    center_mask[int(np.floor(cx)), int(np.floor(cy))] = 0
    center_edt = ndi.distance_transform_edt(center_mask)
    center_edt[np.logical_not(img2d)] = 0
    return center_edt

def edt_bg_from_particle(img2d: np.ndarray) -> np.ndarray:
    assert 2 == len(img2d.shape), img2d.shape
    if 0 == img2d.sum():
        return img2d
    return ndi.distance_transform_edt(img2d == 0)

In [3]:
def binary_dilation_with_fixed_overlap(
    mask: np.ndarray, dilation_radius: int
) -> Tuple[np.ndarray, np.ndarray]:
    print("# EDT")
    distance = ndi.distance_transform_edt(mask)
    print("# MAXIMA")
    coords = peak_local_max(distance,
                            threshold_abs=22,
                            min_distance=20,
                            footprint=square(30))
    peaks_mask = np.zeros(distance.shape, dtype=bool)
    peaks_mask[tuple(coords.T)] = True
    peaks_markers, _ = ndi.label(peaks_mask)
    print("# WATERSHED")
    labels0 = label(mask)
    for i in range(1, labels0.max()+1):
        if 500 > np.sum(labels0 == i):
            labels0[labels0 == i] = 0
    labels = watershed(-distance, peaks_markers, mask=mask)+labels0*100
    # Re-filter size after watersheding
    for i in tqdm(sorted(set(np.unique(labels)))):
        if 500 > np.sum(labels == i):
            labels[labels == i] = 0
    # Relabel to avoid missing labels
    for i, v in tqdm(tqdm(enumerate(sorted(set(np.unique(labels)))))):
        labels[labels == v] = i
    print("# SPLIT MASKS")
    single_masks = np.array([i == labels for i in range(0, labels.max()+1)])
    single_masks[0, :, :] = np.zeros(single_masks.shape[1:])
    print("# FILL")
    single_filled_masks = np.array(Parallel(n_jobs=min(20, cpu_count()), verbose=10)(
        delayed(ndi.binary_fill_holes)(single_masks[i, :, :])
        for i in range(single_masks.shape[0])))
    filled_labels = np.zeros(labels.shape)
    for i in range(single_filled_masks.shape[0]):
        filled_labels += single_filled_masks[i, :, :] * i
    filled_labels = filled_labels.astype("int64")
    print("# DILATE")
    single_dilated_masks = np.array(Parallel(n_jobs=min(20, cpu_count()), verbose=10)(
        delayed(binary_dilation)(single_filled_masks[i, :, :], square(30))
        for i in range(single_filled_masks.shape[0])))
    dilation_sum = single_dilated_masks.sum(0)
    print("# CERTAIN VALUES")
    certain_pixels = np.logical_and(dilation_sum == 1, np.logical_not(filled_labels != 0))
    certain_pixel_values = np.array([single_dilated_masks[i, :, :]*i
                                     for i in tqdm(range(single_dilated_masks.shape[0]))])
    for i in tqdm(range(certain_pixel_values.shape[0])):
        certain_pixel_values[i, np.logical_not(certain_pixels)] = 0
    print("# UNCERTAIN VALUES")
    uncertain_pixels = np.logical_and(dilation_sum > 1, np.logical_not(filled_labels != 0))
    uncertain_coords = np.array(np.where(uncertain_pixels))
    print("# EDT")
    single_edt = np.array(Parallel(n_jobs=min(20, cpu_count()), verbose=10)(
        delayed(edt_bg_from_particle)(single_filled_masks[i, :, :])
        for i in range(single_filled_masks.shape[0])))
    print("# OUTPUT")
    dilated_labels = filled_labels.copy()
    # Fix uncertain pixels
    for i in tqdm(range(uncertain_coords.shape[1])):
        icoords = uncertain_coords.transpose()[i, :]
        overlapping_labels = np.where(single_dilated_masks[:, icoords[0], icoords[1]])[0]
        selected_label = overlapping_labels[np.argmin(
            single_edt[overlapping_labels, icoords[0], icoords[1]])]
        dilated_labels[icoords[0], icoords[1]] = selected_label
    dilated_labels += certain_pixel_values.sum(0)
    return filled_labels, dilated_labels

In [4]:
def plot_dilation(labels: np.ndarray, dilated_labels: np.ndarray, image: np.ndarray) -> None:
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.imshow(image.sum(0)*-1, cmap="binary", interpolation="none")
    for i in tqdm(range(1, labels.max()+1)):
        for contour in find_contours(labels == i, .5):
            ax.plot(contour[:, 1], contour[:, 0], linewidth=1)
        for contour in find_contours(dilated_labels == i, .5):
            ax.plot(contour[:, 1], contour[:, 0], linewidth=1, linestyle='--')
    plt.close(fig)
    return fig, ax

In [5]:
def run_series(series_id: int) -> None:
    mask = imread(f"../../data/60x_mask/original/a647_{series_id:03d}.tif").reshape((2048, 2048))
    image = imread(f"/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_dw/a647_{series_id:03d}.tif")
    labels, dilated_labels = binary_dilation_with_fixed_overlap(mask, 30)
    return labels, dilated_labels, image

In [6]:
for i in range(1, 8):
    print(f">>>>> SERIES {i} <<<<<")
    labels, dilated_labels, image = run_series(i)
    fig, ax = plot_dilation(labels, dilated_labels, image)
    tf.imwrite(f"../../data/60x_mask/labels/a647_{i:03d}.labels.tiff",
               labels, software="GG20210811")
    tf.imwrite(f"../../data/60x_mask/dilated_labels_watershed/a647_{i:03d}.dilated_labels.tiff",
               dilated_labels, software="GG20210811")
    fig.savefig(f"../../data/60x_mask/dilated_labels_watershed_contours/a647_{i:03d}.contours.png")

>>>>> SERIES 1 <<<<<
# EDT
# MAXIMA
# WATERSHED


100%|██████████| 201/201 [00:01<00:00, 126.59it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
24it [00:00, 227.26it/s]
50it [00:00, 234.37it/s]
76it [00:00, 240.20it/s]
101it [00:00, 241.38it/s]
127it [00:00, 246.56it/s]
153it [00:00, 248.36it/s]
182it [00:00, 250.21it/s]
182it [00:00, 250.66it/s]


# SPLIT MASKS
# FILL


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:    2.1s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:    2.3s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:    3.1s
[Parallel(n_jobs=20)]: Done 162 out of 182 | elapsed:    3.4s remaining:    0.4s
[Parallel(n_jobs=20)]: Done 182 out of 182 | elapsed:    3.7s finished


# DILATE


...

100%|██████████| 223/223 [00:07<00:00, 28.39it/s]
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Batch computation too fast (0.0545s.) Setting batch_size=2.


# UNCERTAIN VALUES
# EDT


[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    2.5s
[Parallel(n_jobs=20)]: Batch computation too slow (2.7267s.) Setting batch_size=1.
[Parallel(n_jobs=20)]: Done  50 tasks      | elapsed:    4.2s
[Parallel(n_jobs=20)]: Done  76 tasks      | elapsed:    6.9s
[Parallel(n_jobs=20)]: Done 106 tasks      | elapsed:    9.9s
[Parallel(n_jobs=20)]: Done 128 tasks      | elapsed:   12.0s
[Parallel(n_jobs=20)]: Done 145 tasks      | elapsed:   13.4s
[Parallel(n_jobs=20)]: Done 162 tasks      | elapsed:   14.8s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:   16.6s
[Parallel(n_jobs=20)]: Done 207 out of 223 | elapsed:   18.9s remaining:    1.5s
[Parallel(n_jobs=20)]: Done 223 out of 223 | elapsed:   20.3s finished
  8%|▊         | 9275/123165 [00:00<00:01, 92743.90it/s]

# OUTPUT


100%|██████████| 222/222 [00:25<00:00,  8.56it/s]
